# Email Spam Detection
#### Source can also be from https://www.kaggle.com/code/mfaisalqureshi/email-spam-detection-98-accuracy

### Reference Libraries

In [53]:
#!pip install termcolor # This will help you print text with colors

In [89]:
import pandas as pd # data processing, file operations
import numpy as np
from sklearn.pipeline import Pipeline
from termcolor import colored
import json
import yaml
import sys # appending path to current project
import os # importing to get path

# Build the model
#CounterVectorizer Convert the text into matrics
from sklearn.feature_extraction.text import TfidfVectorizer
# Naive Bayes Have three Classifier(Bernouli,Multinominal,Gaussian) 
# Here we use Multinominal Bayes Because the data is in a discrete form 
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline # KR: What does this pipeline do?
# Training the data
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.svm import SVC
import warnings

warnings.filterwarnings('ignore')

sys.path.append(os.path.abspath('../Libraries'))
from model_info import ModelInfo # from filename without .py import class_name


In [86]:
# defining a class to hold information of the selected model
# class ModelInfo:
    
#     def __init__(self) -> None:
#         self.ModelName = "No Model Selected"
#         self.Model = None
#         self.Tokenizer = None
#         self.AccuracyScore = np.nan

In [90]:
class SpamDetector:
        
    def __load_json_configurations(self):
        with open("../configuration/config.json") as f:
            config = json.load(f)

        self.__configs["enable_trace"] = config["Settings"]["enable_trace"]
        self.__configs["log_level"] = config["Settings"]["log_level"]
        self.__configs["log_path"] = config["Settings"]["log_path"]
        # self.__configs["host"] = config["Database"]["host"]
    
    def __load_yaml_configurations(self):
        with open("../configuration/config.yaml") as f:
            config = yaml.safe_load(f)

        self.__configs["enable_trace"] = config["Settings"]["enable_trace"]
        self.__configs["log_level"] = config["Settings"]["log_level"]
        self.__configs["log_path"] = config["Settings"]["log_path"]
        # self.__configs["host"] = config["Database"]["host"]
        
    def __load_data(self):
        df = pd.read_csv('../data/raw/mail_data.csv')
        return df

    def __extract_transform_load(self, df):
        # Checking for null
        empty_record_total = df.isna().sum()
        print (f"empty_record_total:{empty_record_total}")
        
        # Add a category which is numerical
        df["IsSpam"] = df['Category'].map({'spam': 1, 'ham': 0}) # an alternate way to set the values instead of using lambda
        # df["IsSpam1"] = df['Category'].apply(lambda x:1 if x=='spam' else 0) # We need numerical column for our AI Model
    
    # Defining all the models that i want to try into a dictionary
    # TODO: KR: Read these information from a config file and write a function to load this as a pipe
    # TODO: KR: Make the algorithms dynamic so that we can build a class to add new models without touching the code
    def __get_training_models(self):
        self.__models_dict = {
            "Logistic Regression": LogisticRegression(),
            "Naive Bayes Multinomia": MultinomialNB()
        }
        return self.__models_dict
        
    def __select_best_model(self, df):
        # Loop through the models and for each model you will have to train and test and get the accuracy
        # Since we canot loop and reference via index, we are converting the dictionary to a list
        models_dict = self.__get_training_models()
        models_list = list(models_dict)
        models_values_list = list(models_dict.values())
        temp_average_score = 0.0
        models_info = list()
        selected_model = ModelInfo()
        recomended_model_count = len(models_list)
        trace_enabled = self.__configs['enable_trace']
        
        print(colored(f"self.__configs.enable_trace: {self.__configs['enable_trace']}", "red"))
        
        if (trace_enabled):
            print(f"Recomended Models Count: {recomended_model_count}")
            
        for l in range(len(models_list)):
            print(colored(f"\nCurrent Model: {models_list[l]} [{models_values_list[l]}]", 'yellow'))
            model = models_values_list[l]
            # vectorize the x_train data and x_test data
            vectorizer = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True) # object to tokenize the messages

            # Calculating the mean of the accuracy so that way we know if this would work for various test data
            meanAccuracy_train_data = 0.0
            repeatCount = 20 # Ensure we get the accuracy for different train data
            for i in range(0, repeatCount, 1):
                x_train, x_test, y_train, y_test = train_test_split(df['Message'], df['IsSpam'], test_size=0.20)
                
                x_train_vect = vectorizer.fit_transform(x_train)
                x_test_vect = vectorizer.transform(x_test)

                # fit the vector data into the model
                model.fit(x_train_vect, y_train)
                
                y_pred = model.predict(x_test_vect) # test with vectorized test data

                # calculate the accuracy score for the model
                accuracyScore = accuracy_score(y_test, y_pred)
                if (trace_enabled):
                    print (f'\t The accuracy score for the model {model} iteration {i} is {accuracyScore}')
                meanAccuracy_train_data += accuracyScore

                # Predicting
                # y_train_pred = pipe.predict

            meanAccuracy_train_data = meanAccuracy_train_data / repeatCount
            print(colored(f'Mean Accuracy Score [{repeatCount} Iterations]: {meanAccuracy_train_data}', 'magenta'))
            
            if (trace_enabled):
                print(f"\tcurrent_model.AccuracyScore: {meanAccuracy_train_data}, \n\tcurrent_model.ModelName: {models_list[l]}, \n\tcurrent_model.Model: {model}, \n\tcurrent_model.Tokenizer: {vectorizer}")

            current_model = ModelInfo()
            current_model.AccuracyScore = meanAccuracy_train_data
            current_model.ModelName = models_list[l]
            current_model.Model = model
            current_model.Tokenizer = vectorizer
            models_info.append(current_model)
            
            if (trace_enabled):
                print(f"\tmodels_info_count: {len(models_info)}, models_info_type: {type(models_info)}")
                print(f"\t index:{l}, {models_info[l].Model}")
                #print(f"\t index:{l+1}, {models_info[l+1].Model}")
            sel_model_accuracy = np.nan_to_num(selected_model.AccuracyScore)
            
            #print(f"Selected Model's Accuracy:{sel_model_accuracy}, Current Model's Accuracy:{current_model.AccuracyScore}")
            if (sel_model_accuracy < current_model.AccuracyScore):
                selected_model = current_model

        # if the accuracy score is mre than the other ones, than we need to take the best algorithm to predict.
        for m in models_info:
            print(f'Model Name:{m.ModelName} with accuracy {m.AccuracyScore}')
            
        return selected_model
    
    def IsSpam(self, message):
        input_data_features = self.selected_model.Tokenizer.transform([message])

        output = self.selected_model.Model.predict(input_data_features)
        result_code = output[0]
        if (1 == result_code):
            result_text = "Spam"
        else:
            result_text = "Not a Spam"
        print(f"The prediction by the algorithm {x.selected_model.Model} is {result_text} [{result_code}]")
        
        return result_code, result_text
        # response_data = {
        #     "code": result_code,
        #     "description": result_text
        # }
        
        # return json.dump(response_data)
        
        
    def __init__ (self) -> None:
        self.__configs = {
                "enable_trace": False,
                "log_level": "DEBUG",
                "log_path": "."
            }
        # load configuration files
        self.__load_json_configurations()
        
        self.__df = self.__load_data()
        self.__extract_transform_load(self.__df)
        
        # Selecting the best model based on the data that was given
        self.selected_model = self.__select_best_model(self.__df)
        print(f"\n The selected Model's Name: {self.selected_model.ModelName}")

In [91]:
x = SpamDetector()
x.selected_model.ModelName



empty_record_total:Category    0
Message     0
dtype: int64
self.__configs.enable_trace: False

Current Model: Logistic Regression [LogisticRegression()]
Mean Accuracy Score [20 Iterations]: 0.9638116591928252

Current Model: Naive Bayes Multinomia [MultinomialNB()]
Mean Accuracy Score [20 Iterations]: 0.9696860986547087
Model Name:Logistic Regression with accuracy 0.9638116591928252
Model Name:Naive Bayes Multinomia with accuracy 0.9696860986547087

 The selected Model's Name: Naive Bayes Multinomia


'Naive Bayes Multinomia'

### Prepare the training set

In [84]:
res = x.IsSpam("Free entry in 2 a wkly comp to win FA Cup")
res

The prediction by the algorithm MultinomialNB() is Spam [1]


(1, 'Spam')